# Simple Doc2Vec

- Standard preprocessing
- One network per grounding Document
- Predict the best matching span
- Use RCQuestion set as is (give in question with history, no processing of question other than the same prepocessing)

In [ ]:
# 1. Train models
#    - pre process document data -> Grounding Docs
#    - tune model parameters -> Batch Trainer
#    DATA: Documents dataset
#



# 2. Predict rc question
#    - pre process rc question -> RCData
#    - predict answers and create predictions and references -> BatchTrainer
#    DATA: RC dataset
#


# 3. Evaluate results
#    - Squad 2 metrics -> Evaluate

